<a href="https://colab.research.google.com/github/yhc-666/MultiagentLogic/blob/master/scripts/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/yhc-666/MultiagentLogic.git
%cd MultiagentLogic

fatal: destination path 'MultiagentLogic' already exists and is not an empty directory.
/content/MultiagentLogic


In [1]:
from src.symbolic_solvers.fol_solver.prover9_solver import FOL_Prover9_Program
from src.symbolic_solvers.pyke_solver.pyke_solver import Pyke_Program
from src.symbolic_solvers.z3_solver.sat_problem_solver import LSAT_Z3_Program










ModuleNotFoundError: No module named 'src'

In [3]:
!pip install -r requirements.txt

## Pyke Solver

In [4]:
sample_lp_program = "Predicates:\nCold($x, bool) ::: Is x cold?\nQuiet($x, bool) ::: Is x quiet?\nRed($x, bool) ::: Is x red?\nSmart($x, bool) ::: Is x smart?\nKind($x, bool) ::: Is x kind?\nRough($x, bool) ::: Is x rough?\nRound($x, bool) ::: Is x round?\nFacts:\nCold(Bob, True) ::: Bob is cold.\nQuiet(Bob, True) ::: Bob is quiet.\nRed(Bob, True) ::: Bob is red.\nSmart(Bob, True) ::: Bob is smart.\nKind(Charlie, True) ::: Charlie is kind.\nQuiet(Charlie, True) ::: Charlie is quiet.\nRed(Charlie, True) ::: Charlie is red.\nRough(Charlie, True) ::: Charlie is rough.\nCold(Dave, True) ::: Dave is cold.\nKind(Dave, True) ::: Dave is kind.\nSmart(Dave, True) ::: Dave is smart.\nQuiet(Fiona, True) ::: Fiona is quiet.\nRules:\nQuiet($x, True) && Cold($x, True) >>> Smart($x, True) ::: If something is quiet and cold then it is smart.\nRed($x, True) && Cold($x, True) >>> Round($x, True) ::: Red, cold things are round.\nKind($x, True) && Rough($x, True) >>> Red($x, True) ::: If something is kind and rough then it is red.\nQuiet($x, True) >>> Rough($x, True) ::: All quiet things are rough.\nCold($x, True) && Smart($x, True) >>> Red($x, True) ::: Cold, smart things are red.\nRough($x, True) >>> Cold($x, True) ::: If something is rough then it is cold.\nRed($x, True) >>> Rough($x, True) ::: All red things are rough.\nSmart(Dave, True) && Kind(Dave, True) >>> Quiet(Dave, True) ::: If Dave is smart and Dave is kind then Dave is quiet.\nQuery:\nKind(Charlie, True) ::: Charlie is kind."

In [5]:
pyke_program = Pyke_Program(sample_lp_program, 'ProofWriter')
result, error_message, reasoning = pyke_program.execute_with_reasoning()

In [6]:
print(reasoning)

Complete Reasoning Path:
We first define following predicates and corresponding natural language explanations:
  Cold($x, bool) ::: Is x cold?
  Quiet($x, bool) ::: Is x quiet?
  Red($x, bool) ::: Is x red?
  Smart($x, bool) ::: Is x smart?
  Kind($x, bool) ::: Is x kind?
  Rough($x, bool) ::: Is x rough?
  Round($x, bool) ::: Is x round?
We have following known facts from the context:
  Cold(Bob, True) ::: Bob is cold.
  Quiet(Bob, True) ::: Bob is quiet.
  Red(Bob, True) ::: Bob is red.
  Smart(Bob, True) ::: Bob is smart.
  Kind(Charlie, True) ::: Charlie is kind.
  Quiet(Charlie, True) ::: Charlie is quiet.
  Red(Charlie, True) ::: Charlie is red.
  Rough(Charlie, True) ::: Charlie is rough.
  Cold(Dave, True) ::: Dave is cold.
  Kind(Dave, True) ::: Dave is kind.
  Smart(Dave, True) ::: Dave is smart.
  Quiet(Fiona, True) ::: Fiona is quiet.
We have following known rules from the context:
  rule1: Quiet($x, True) && Cold($x, True) >>> Smart($x, True)
  rule2: Red($x, True) && Cold

## Prove9 Solver

### Case1: Answer is Ture

In [7]:
logic_program_t = """Premises:
    ∀x (Drinks(x) → Dependent(x)) ::: All people who regularly drink coffee are dependent on caffeine.
    ∀x (Drinks(x) ⊕ Jokes(x)) ::: People either regularly drink coffee or joke about being addicted to caffeine.
    ∀x (Jokes(x) → ¬Unaware(x)) ::: No one who jokes about being addicted to caffeine is unaware that caffeine is a drug.
    (Student(rina) ∧ Unaware(rina)) ⊕ ¬(Student(rina) ∨ Unaware(rina)) ::: Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug.
    ¬(Dependent(rina) ∧ Student(rina)) → (Dependent(rina) ∧ Student(rina)) ⊕ ¬(Dependent(rina) ∨ Student(rina)) ::: If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    Conclusion:
    Jokes(rina) ⊕ Unaware(rina) ::: Rina is either a person who jokes about being addicted to caffeine or is unaware that caffeine is a drug.
    """

In [8]:
prover9_program = FOL_Prover9_Program(logic_program_t)

In [9]:
result, error_message, reasoning = prover9_program.execute_program()
print(f"result: {result}")

result: True


In [10]:
print(reasoning)

prove original conclusion:
1 (all x (Drinks(x) -> Dependent(x))).  [assumption].
2 (all x (Drinks(x) & -Jokes(x) | -Drinks(x) & Jokes(x))).  [assumption].
3 (all x (Jokes(x) -> -Unaware(x))).  [assumption].
4 Student(Rina) & Unaware(Rina) & --(Student(Rina) | Unaware(Rina)) | -(Student(Rina) & Unaware(Rina)) & -(Student(Rina) | Unaware(Rina)).  [assumption].
5 (-(Dependent(Rina) & Student(Rina)) -> Dependent(Rina) & Student(Rina)) & --(Dependent(Rina) | Student(Rina)) | -(-(Dependent(Rina) & Student(Rina)) -> Dependent(Rina) & Student(Rina)) & -(Dependent(Rina) | Student(Rina)).  [assumption].
6 Jokes(Rina) & -Unaware(Rina) | -Jokes(Rina) & Unaware(Rina).  [goal].
7 Drinks(x) | Jokes(x).  [clausify(2)].
8 -Drinks(x) | Dependent(x).  [clausify(1)].
10 Jokes(Rina) | -Unaware(Rina).  [deny(6)].
11 -Jokes(x) | -Unaware(x).  [clausify(3)].
12 -Jokes(Rina) | Unaware(Rina).  [deny(6)].
13 Jokes(x) | Dependent(x).  [resolve(7,a,8,a)].
14 Unaware(Rina) | -Student(Rina).  [clausify(4)].
17 Stude

### Case2: Answer is fasle

In [11]:
logic_program_f = """Premises:
    Czech(miroslav) ∧ ChoralConductor(miroslav) ∧ Specialize(miroslav, renaissance) ∧ Specialize(miroslav, baroque) ::: Miroslav Venhoda was a Czech choral conductor who specialized in the performance of Renaissance and Baroque music.
    ∀x (ChoralConductor(x) → Musician(x)) ::: Any choral conductor is a musician.
    ∃x (Musician(x) ∧ Love(x, music)) ::: Some musicians love music.
    Book(methodOfStudyingGregorianChant) ∧ Author(miroslav, methodOfStudyingGregorianChant) ∧ Publish(methodOfStudyingGregorianChant, year1946) ::: Miroslav Venhoda published a book in 1946 called Method of Studying Gregorian Chant.
    Conclusion:
    ¬∃x (ChoralConductor(x) ∧ Specialize(x, renaissance)) ::: No choral conductor specialized in the performance of Renaissance.
    """

In [12]:
prover9_program = FOL_Prover9_Program(logic_program_f)

In [13]:
result, error_message, reasoning = prover9_program.execute_program()
print(f"result: {result}")

result: False


In [14]:
print(reasoning)

prove negation of original conclusion:
1 Czech(Miroslav) & ChoralConductor(Miroslav) & Specialize(Miroslav,Renaissance) & Specialize(Miroslav,Baroque).  [assumption].
5 --(exists x (ChoralConductor(x) & Specialize(x,Renaissance))).  [goal].
7 ChoralConductor(Miroslav).  [clausify(1)].
8 -ChoralConductor(x) | -Specialize(x,Renaissance).  [deny(5)].
9 -Specialize(Miroslav,Renaissance).  [resolve(8,a,7,a)].
10 Specialize(Miroslav,Renaissance).  [clausify(1)].
12 $F.  [resolve(9,a,10,a)].


### Case3: Answer is unkonw

In [19]:
logic_program_unk = """Premises:
    ∀x (TalentShows(x) → Engaged(x)) ::: If people perform in school talent shows often, then they attend and are very engaged with school events.
    ∀x (TalentShows(x) ∨ Inactive(x)) ::: People either perform in school talent shows often or are inactive and disinterested members of their community.
    ∀x (Chaperone(x) → ¬Students(x)) ::: If people chaperone high school dances, then they are not students who attend the school.
    ∀x (Inactive(x) → Chaperone(x)) ::: All people who are inactive and disinterested members of their community chaperone high school dances.
    ∀x (AcademicCareer(x) → Students(x)) ::: All young children and teenagers who wish to further their academic careers and educational opportunities are students who attend the school.
    Conclusion:
    TalentShows(bonnie) ::: Bonnie performs in school talent shows often.
    """

In [20]:
prover9_program = FOL_Prover9_Program(logic_program_unk)

In [21]:
result, error_message, reasoning = prover9_program.execute_program()
print(f"result: {result}")

result: Unknown


In [22]:
print(reasoning)

trying to prove original conclusion:
1 (all x (TalentShows(x) -> Engaged(x))) # label(non_clause). [assumption].
2 (all x (TalentShows(x) | Inactive(x))) # label(non_clause). [assumption].
3 (all x (Chaperone(x) -> -Students(x))) # label(non_clause). [assumption].
4 (all x (Inactive(x) -> Chaperone(x))) # label(non_clause). [assumption].
5 (all x (AcademicCareer(x) -> Students(x))) # label(non_clause). [assumption].
6 TalentShows(Bonnie) # label(non_clause) # label(goal). [goal].
7 -TalentShows(x) | Engaged(x). [clausify(1)].
8 TalentShows(x) | Inactive(x). [clausify(2)].
9 -Chaperone(x) | -Students(x). [clausify(3)].
10 -Inactive(x) | Chaperone(x). [clausify(4)].
11 -AcademicCareer(x) | Students(x). [clausify(5)].
12 -TalentShows(Bonnie). [deny(6)].
13 Derived: Inactive(x) | Engaged(x). [resolve(8,a,7,a)].
14 Derived: Inactive(Bonnie). [resolve(12,a,8,a)].
15 Derived: -Inactive(x) | -Students(x). [resolve(10,b,9,a)].
16 Derived: -Students(x) | Engaged(x). [resolve(15,a,13,a)].
17 Deri

## SAT solver 

### Case1: Execution successful

### Case2: Execution unsuccessful